# **Introduction**

Welcome to the programming portion of assignment 2 of CS 4756/5756. In this assignment, you will implement several variations of Policy Gradient methods. Concretely, you will:
* Implement the REINFORCE algorithm (Part 1)
* Incorporate Reward-To-Go into the REINFORCE loss function (Part 2)

You will use the FetchReach-v4 environment for this assignment. Refer to the Gymnasium-Robotics website for more details about [this environment](https://robotics.farama.org/envs/fetch/reach/).


Please read through the following paragraphs carefully.

**Getting Started:** You should complete this assignment on **[Google Colab](https://colab.research.google.com/)**.

**Evaluation:**
Your code will be tested for correctness and, for certain assignments, speed. For this particular assignment, performance results will not be harshly graded (although we provide approximate expected reward numbers, you are not expected to replicate them exactly). Please remember that all assignments should be completed individually.

**Academic Integrity:** We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else’s code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don’t try. We trust you all to submit your own work only; please don’t let us down. If you do, we will pursue the strongest consequences available to us.

**Getting Help:** The [Resources](https://www.cs.cornell.edu/courses/cs4756/2025sp/#resources) section on the course website is your friend! If you ever feel stuck in these projects, please feel free to avail yourself to office hours and Edstem! If you are unable to make any of the office hours listed, please let TAs know and we will be happy to assist. If you need a refresher for PyTorch, please see this [60 minute blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)! For Numpy, please see the quickstart [here](https://numpy.org/doc/stable/user/quickstart.html) and full API [here](https://numpy.org/doc/stable/reference/).


# **Setup**

**Please run the cells below to install the necessary packages**.



In [ ]:
import sys
USING_COLAB = 'google.colab' in sys.modules

if USING_COLAB:
  !apt-get -qq update
  !apt-get -qq install -y libosmesa6-dev libgl1-mesa-glx libglfw3 libgl1-mesa-dev libglew-dev patchelf
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
else:
  !pip install torch torchvision torchaudio
  !pip install numpy
  !pip install tqdm
  !pip install opencv-python

!pip install matplotlib
!pip install -U mediapy
!pip install -U renderlab
!pip install -U "imageio<3.0"
!git clone https://github.com/Farama-Foundation/Gymnasium-Robotics.git
!pip install -e Gymnasium-Robotics
sys.path.append('/content/Gymnasium-Robotics')

In [ ]:
import os
# Mujoco GLEW Setup
try:
  if _mujoco_run_once:
    pass
except NameError:
  _mujoco_run_once = False
if not _mujoco_run_once:
  try:
    os.environ['LD_PRELOAD']=os.environ['LD_PRELOAD'] + ':/usr/lib/x86_64-linux-gnu/libGLEW.so'
  except KeyError:
    os.environ['LD_PRELOAD']='/usr/lib/x86_64-linux-gnu/libGLEW.so'
  # presetup so we don't see output on first env initialization
  _mujoco_run_once = True
  if USING_COLAB:
    NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
    if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
      with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
          f.write("""{
          "file_format_version" : "1.0.0",
          "ICD" : {
              "library_path" : "libEGL_nvidia.so.0"
          }
      }
      """)
  # Set environment variable to support EGL (off-screen) rendering
  %env MUJOCO_GL=egl

# Import packages and initial seeding


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.distributions as D
import torch.optim as optim
import gymnasium as gym
import gymnasium_robotics
import gymnasium.wrappers as wrappers
import random
import tqdm
import matplotlib.pyplot as plt

In [ ]:
seed = 695

# Setting the seed to ensure reproducability
def reseed(seed : int):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

reseed(seed)

In [ ]:
# In this block we define wrappers necessary to simplify the environment MDP
def wrap_reach_fixed_goal(env):
  g = np.array([1.486, 0.73, 0.681], dtype=np.float32)
  env.unwrapped._sample_goal = lambda: g
  return env

class FetchRewardWrapper(gym.Wrapper):
  def reset(self, *args, **kwargs):
    obs, info = self.env.reset(*args, **kwargs)
    self.prev_dist = np.linalg.norm(obs['achieved_goal'] - obs['desired_goal'])
    return obs, info

  def step(self, action):
    obs, reward, terminated, truncated, info = self.env.step(action)

    achieved_goal = obs['achieved_goal']
    desired_goal = obs['desired_goal']
    current_dist = np.linalg.norm(achieved_goal - desired_goal)
    reward = (self.prev_dist - current_dist) * 10
    self.prev_dist = current_dist
    return obs, reward, terminated, truncated, info

# Visualize Helper Function

Below, we provide the helper function `visualize` for your use. This function will create a visualization of the environment passed in the parameter `env`. If you are using Colab, calling this function will render the visualization within the notebook. If you are using your local machine, this function will instead save a video of the visualization to your current directory (rendering videos in Jupyter Notebooks is not widely supported outside of Colab).

In [ ]:
def visualize(env : gym.Env, algorithm=None, video_name="test"):
    """Visualize a policy network for a given algorithm on a single episode

        Args:
            algorithm (PolicyGradient): Algorithm whose policy network will be rolled out for the episode. If
            no algorithm is passed in, a random policy will be visualized.
            video_name (str): Name for the mp4 file of the episode that will be saved (omit .mp4). Only used
            when running on local machine.
    """

    def get_action(obs):
        if not algorithm:
            return env.action_space.sample()
        else:
            return algorithm.compute_action(obs)

    if USING_COLAB:
        import renderlab as rl

        directory = './video'
        env = rl.RenderFrame(env, "output/")
        obs, info = env.reset()
        for i in range(500):
            action = get_action(obs)
            obs, reward, terminated, truncated, info = env.step(action)

            if terminated or truncated:
                break

        env.play()
    else:
        import cv2

        video = cv2.VideoWriter(f"{video_name}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 24, (600,400))
        obs = env.reset()
        for i in range(500):
            action = get_action(obs)
            obs, reward, terminated, truncated, info = env.step(action)

            if terminated or truncated:
                break

            im = env.render(mode='rgb_array')
            im = im[:,:,::-1]

            video.write(im)

        video.release()
        env.close()
        print(f"Video saved as {video_name}.mp4")

## **Notes about Fetch Reach Environment**
The environment uses a Fetch Robot, which is a 7-DoF Mobile Manipulator.

The task is a *goal-reaching task*:
The observation space contains '`observation`' which includes the state of the robot in the environment, and '`desired_goal`' which specifies the xyz coordinate that the robot's gripper aims to reach.

See https://robotics.farama.org/envs/fetch/reach/ for more details.


If the goal is reached, `info['is_success']` will be set to 1, and this is an indication that we should terminate the rollout.


The reward is -1 per timestep spent in the environment without completing the task, with 50 steps being the limit (so -50 is the worst episode return).

> Note: For this assignment, we've modified the environment that it only has a fixed goal to reach, and has better reward shaping, since vanilla REINFORCE will produce very noisy gradients for complex markov chains.

**Run the cell below to create and visualize the environment:**

In [ ]:
# Let's initialize the environment first
reseed(seed)
env = gym.make("FetchReach-v4", render_mode="rgb_array")
env = wrap_reach_fixed_goal(env)
env = FetchRewardWrapper(env)
env = wrappers.FilterObservation(env, ["desired_goal", "observation"])
env = wrappers.FlattenObservation(env)

In [ ]:
visualize(env)

# Part 1: Vanilla REINFORCE

In this assignment, we will implement the REINFORCE algorithm and apply it to our environment. This algorithm is a popular reinforcement learning algorithm that can be used for both discrete and continuous action spaces.

## Overview

The REINFORCE algorithm uses a neural network to learn a policy that maps states to actions. The algorithm collects a set of trajectories, which are used to update the policy network. Below, we present a brief overview of REINFORCE and the equations behind it. **See MACRL 11.4 for a full description of REINFORCE and derivations for the following equations.**

To start, recall the reinforcement learning objective:
$$ J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}}[r(\tau)] $$


The goal of REINFORCE is to maximize $J(\theta)$. As REINFORCE is a policy gradient algorithm, it involves taking the gradient of this objective with respect to the parameters $\theta$ of the policy $\pi_{\theta}$:

$$ \nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}}[\nabla_θ log \pi_\theta (\tau) r(\tau)] $$

The REINFORCE algorithm approximates this quantity from N trajectories as follows:

$$ \nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^{N} \Biggl( \sum_{t=0}^{T-1} \nabla_\theta log \pi_\theta(a_{it}|s_{it})\Biggl) \Biggl(\sum_{t=0}^{T-1} r(s_{it}, a_{it}) \Biggl) $$

**Note:** You will implement a slightly modified version of REINFORCE that uses the total discounted reward (with discount factor $\gamma$) rather than the total reward in order to encourage the agent to prioritize more immediate rewards over those in the distant future:

$$ \nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^{N} \sum_{t=0}^{T-1} \nabla_\theta log \pi_\theta(a_{it}|s_{it}) R_i $$

where $R_i$ is the total discounted reward for trajectory $i$:

$$ R_i = \sum_{t=0}^{T-1} \gamma^{t} r(s_{it}, a_{it}) $$


As the goal of REINFORCE is to maximize $J(\theta)$, our loss function $L(\theta)$ (which we will be minimizing) will be the following:

$$ L(\theta) = - \frac{1}{N} \sum_{i=1}^{N} \sum_{t=0}^{T-1} log \pi_\theta(a_{it}|s_{it}) R_i$$


## Instructions

You will need to implement the following:

1. `PolicyNet` class - This class will define the policy network used in the REINFORCE algorithm.

The policy will be a MLP network that outputs a **Normal** distribution for a given state:

$$\pi(a_t | s_t) \sim N(\mu(s_t), \sigma(s_t)) $$
where

$$\mu(s_t), \sigma(s_t)$$

are the outputs of the `PolicyNet`.

2. `PolicyGradient` class - This class will define the REINFORCE algorithm.


Follow the instructions below to implement each of these components.

### `PolicyNet` class

The `PolicyNet` class should define a neural network that takes in a state and outputs a probability distribution over the action space. The network should have the following architecture:

- Input layer: a fully-connected layer with `state_dim` input nodes and `hidden_dim` output nodes, followed by a ReLU activation function.

- Output layer: a fully-connected layer with `hidden_dim` input nodes and `action_dim * 2` output nodes (first half for $\mu(s_t)$, second half for $\log(\sigma(s_t))$).


Finally, after the output layer, return a `torch.distributions.normal.Normal` object with the proper $\mu(s_t)$ and $\sigma(s_t)$.

[Documentation on `torch.distributions.normal.Normal`](https://pytorch.org/docs/stable/distributions.html#normal)

> You should use the `nn` module of PyTorch to define this network, and use `D.Normal` to construct the final return distribution.


In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, state_dim: int, action_dim: int, hidden_dim: int):
        """Policy network for the REINFORCE algorithm.

        Args:
            state_dim (int): Dimension of the state space.
            action_dim (int): Dimension of the action space.
            hidden_dim (int): Dimension of the hidden layers.
        """
        super(PolicyNet, self).__init__()
        # TODO: Implement the policy network for the REINFORCE algorithm here



    def forward(self, state: torch.Tensor) -> D.Normal:
        """Forward pass of the policy network.

        Args:
            state (torch.Tensor): State of the environment. Shape (N, state_dim)


        Returns:
            action_dist (D.Normal): Normal distribution representing \pi(a_t | s_t)
        """
        # TODO: Implement the forward pass of the policy network here



### `PolicyGradient` class

The `PolicyGradient` class should define the REINFORCE algorithm.
The `PolicyGradient` class should have the following methods:

- `__init__(self, env, policy_net, seed, reward_to_go: bool = False)`: Constructor method that initializes the environment and policy network.

- `compute_action(self, state)`: Method that computes an action based on the policy network.

- `compute_loss(self, episode, gamma)`: Method that computes the loss for a given episode.

- `update_policy(self, episodes, optimizer, gamma)`: Method that updates the policy network.

In [ ]:
class PolicyGradient:
    def __init__(
      self,
      policy_net : PolicyNet,
      reward_to_go: bool = False
    ):
      """Policy gradient algorithm based on the REINFORCE algorithm.

      Args:
          policy_net (PolicyNet): Policy network
          reward_to_go (bool): True if using reward_to_go, False if not (False in part 1, True in part 2)
      """
      self.policy_net = policy_net
      self.reward_to_go = reward_to_go

    @property
    def device(self):
      return next(iter(self.policy_net.parameters())).device

    def to(self, device : str | torch.device):
      self.policy_net.to(device)
      return self

    def compute_action(self, state : np.ndarray) -> np.ndarray:
      """Select an action based on the policy network

      Args:
          state (np.ndarray): State of the environment

      Returns:
          action (np.ndarray): Action to take
      """
      with torch.no_grad():
        # TODO: Implement the action selection here based on the policy network output probabilities
        # You also need to convert between numpy arrays and torch tensors


    def compute_loss(
      self,
      episode : list[tuple[
          np.ndarray, np.ndarray, float
      ]],
      gamma : float
    ) -> torch.Tensor:
      """Compute the loss function J for the REINFORCE algorithm

      Args:
          episode (list): List of tuples (state, action, reward)
          gamma (float): Discount factor

      Returns:
          loss (torch.Tensor): The value of the loss function
      """
      # TODO: Extract states, actions and rewards from the episode, and maybe convert them to torch
      if not self.reward_to_go:
        # TODO: Part 1: Remove the following line that starts with "raise", and compute the total discounted reward here
        raise NotImplementedError("Please implement part 1")

      else:
        # TODO: Part 2: Remove the following line that starts with "raise", and compute the discounted rewards to go here
        raise NotImplementedError("Please implement part 2")

      # TODO: Implement the loss function for the REINFORCE algorithm here

      return loss

    def update_policy(self, episodes, optimizer, gamma):
      """Update the policy network using the batch of episodes

      Args:
          episodes (list): List of episodes
          optimizer (torch.optim): Optimizer
          gamma (float): Discount factor
      Returns:
          loss (float): The value of the loss function
      """
      # TODO: Compute the loss function for each episode using compute_loss


      # TODO: Update the policy network using average loss across the batch


      # TODO: Return the loss value


In [ ]:
def collect_episode(env : gym.Env, policy : PolicyGradient | None = None, seed : int | None = None):
  """
  Run an episode of the environment and return the episode

  Returns:
      episode (list): List of tuples (state, action, reward)
  """
  state, info = env.reset(seed=seed)
  episode = []
  done = False
  while not done:
    if policy is not None:
      action = policy.compute_action(state)
    else:
      action = env.action_space.sample()
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    episode.append((state, action, reward))
    state = next_state
  return episode

In [ ]:
def evaluate(env : gym.Env, policy : PolicyGradient | None = None, num_episodes = 100):
  """Evaluate the policy network by running multiple episodes.

  Args:
      env (gym.Env): Environment to evaluate the policy network on
      policy (PolicyGradient): Policy network to evaluate
      num_episodes (int): Number of episodes to run

  Returns:
      average_reward (float): Average total reward per episode
  """
  episode_rewards = []
  for i in range(num_episodes):
    episode = collect_episode(env, policy)
    episode_rewards.append(sum([reward for _, _, reward in episode]))
  return np.mean(episode_rewards)

def train(env : gym.Env, policy : PolicyGradient, num_iterations : int, batch_size : int, gamma : float, lr : float) -> None:
  """Train the policy network using the REINFORCE algorithm

  Args:
      num_iterations (int): Number of iterations to train the policy network
      batch_size (int): Number of episodes per batch
      gamma (float): Discount factor
      lr (float): Learning rate
  """
  policy.policy_net.train()
  optimizer = optim.Adam(policy.policy_net.parameters(), lr=lr)
  losses = []
  # Update the policy every iteration, and use one batch per iteration.
  # Also append computed losses (return values of `update_policy`) to `losses`
  progress = tqdm.trange(num_iterations)
  for iter_i in progress:
    # TODO: Implement the training loop for the REINFORCE algorithm here and add the loss of this iteration to the `losses` array

    losses.append(loss)
    progress.set_postfix(loss=loss)

  return losses

Now that you have implemented REINFORCE, it is time to train and evaluate a policy. Below, we provide training hyperparameters which you should use in your experiments. See the writeup for additional instructions on what metrics and figures you need to include in your submission.

* Expected Training Time (Colab CPU): 15 minutes
* Expected Episodic Reward: 0.5-0.6

In [ ]:
print("Pre-training average return", evaluate(env))

In [ ]:
reseed(seed)
env.reset(seed=seed)
env.action_space.seed(seed)
env.observation_space.seed(seed)

print("Observation Space", env.observation_space)
print("Action Space", env.action_space)

In [ ]:
policy_net = PolicyNet(env.observation_space.shape[-1], env.action_space.shape[-1], 128)
reinforce = PolicyGradient(policy_net, reward_to_go=False)
device = "cpu"
if torch.cuda.is_available():
  device = "cuda"
elif torch.backends.mps.is_available():
  device = "mps"
reinforce.to(device)

print(f"Training on device {device}")

In [ ]:
losses = train(env, reinforce, num_iterations=300, batch_size=16, gamma=0.99, lr=1e-3)

In [ ]:
evaluate(env, reinforce)

Let's now plot the losses we get during the training process.

In [ ]:
plt.plot(losses)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

Why did the loss go up?

- At the start of training, by taking random actions the policy mostly performs random actions and receives mostly negative rewards, when $R_i$ is negative, our loss will be negative since `log_prob` will always be `<= 0`.
  - Ask yourself: what kind of gradient is being propagated when the losses are all negative?
  - The answer is that we're pushing down (minimizing log-likelihood) on all actions we've been executing so far. However, we're minimizing log-likelihood more on actions that produce smaller $R_i$.
- As we optimize this negative loss, we will start to see some positive rewards, and with a positive $R_i$ our loss will be positive.
  - Now think about what kind of gradient is being propagated when the losses are all positive?
  - We're pushing up (maximizing log-likelihood) on all actions that producing positive $R_i$s. However, we're pushing up harder on actions that produce bigger $R_i$.

A very natural thing to think is that can we add a baseline function $b(s_t)$ (ideally, the **mean** value of $R_i$ of following the policy distribution) so that

- For actions better than the mean value $b(s_t)$, we can **push up** the density function of the action distribution
- For actions worse than the mean value $b(s_t)$, we can **push down** the density function of the action distribution

We're effectively reducing the variance on the gradient of the policy. This will be further discussed in the next assignment when you work on actor-critic algorithms.



Now let's save the policy and visualize it!

In [ ]:
# Let's save and visualize our policy
torch.save(reinforce.policy_net.state_dict(), "reinforce.pt")
visualize(env, algorithm=reinforce, video_name="reinforce")

# Part 2: REINFORCE with Reward-to-Go

In this part of the assignment, we will modify the REINFORCE algorithm to use the (discounted) reward-to-go instead of the total discounted reward.

To decrease the variance of the policy gradient, one approach is to make use of causality by observing that the policy cannot influence past rewards. This results in a revised objective where the total rewards only include those acquired after the policy is evaluated. These rewards are considered a sample estimation of the Q function and are known as the "reward-to-go".

## Overview

The reward-to-go is defined as:

$$ R_t = \sum_{t'=t}^{T-1} \gamma^{t'-t} r(s_{t'}, a_{t'}) $$


The updated policy gradient will look like this (same as before, except discounted reward-to-go instead of discounted total reward):

$$ \nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^{N}  \sum_{t=0}^{T-1} \nabla_\theta log \pi_\theta(a_{it}|s_{it}) R_{it} $$

Thus, the updated loss function will be:

$$ L(\theta) = - \frac{1}{N} \sum_{i=1}^{N} \sum_{t=0}^{T-1} log \pi_\theta(a_{it}|s_{it}) R_{it}$$




## Instructions

Modify the `compute_loss` method to allow for the use of reward-to-go instead of the total discounted reward.

Now that you have implemented REINFORCE with reward-to-go, it is time to train and evaluate a policy. Below, we provide training hyperparameters which you should use in your experiments. See the writeup for additional instructions on what metrics and figures you need to include in your submission.

* Expected Training Time (Colab CPU): 15 minutes
* Expected Reward (when calling `evaluate(100)`): 0.9-1.0

In [ ]:
reseed(seed)
env.reset(seed=seed)
env.action_space.seed(seed)
env.observation_space.seed(seed)

print("Observation Space", env.observation_space)
print("Action Space", env.action_space)

In [ ]:
policy_net = PolicyNet(env.observation_space.shape[-1], env.action_space.shape[-1], 128)
reinforce = PolicyGradient(policy_net, reward_to_go=True)
reinforce.to(device)

reinforce.policy_net.load_state_dict(torch.load("reinforce.pt", map_location=device, weights_only=True))
print(f"Training on device {device}")

In [ ]:
losses = train(env, reinforce, num_iterations=300, batch_size=16, gamma=0.99, lr=5e-4)

In [ ]:
plt.plot(losses)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

In [ ]:
evaluate(env, reinforce)

In [ ]:
visualize(env, reinforce)